## Building a Chatbot

Conversational RAG: Enable a chatbot experience over an external source of data.<br/>
Agents: Build a chatbot that can take actions

In [88]:
import os
from dotenv import load_dotenv
load_dotenv() # loading all the envioronment variables

groq_api_key=os.getenv("GROQ_API_KEY")


In [24]:
from langchain_groq import ChatGroq

model=ChatGroq(model="llama-3.1-8b-instant",groq_api_key=groq_api_key)
model

ChatGroq(profile={'max_input_tokens': 131072, 'max_output_tokens': 8192, 'image_inputs': False, 'audio_inputs': False, 'video_inputs': False, 'image_outputs': False, 'audio_outputs': False, 'video_outputs': False, 'reasoning_output': False, 'tool_calling': True}, client=<groq.resources.chat.completions.Completions object at 0x0000019D2BB76D70>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x0000019D2BAFAD40>, model_name='llama-3.1-8b-instant', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [25]:
from langchain_core.messages import HumanMessage
model.invoke([HumanMessage(content="Hi, My name is Anjali")])



AIMessage(content="Hello Anjali, it's nice to meet you. Is there something I can help you with or would you like to chat?", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 28, 'prompt_tokens': 43, 'total_tokens': 71, 'completion_time': 0.023516672, 'completion_tokens_details': None, 'prompt_time': 0.006005388, 'prompt_tokens_details': None, 'queue_time': 0.055095532, 'total_time': 0.02952206}, 'model_name': 'llama-3.1-8b-instant', 'system_fingerprint': 'fp_ff2b098aaf', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None, 'model_provider': 'groq'}, id='lc_run--019c2254-0604-7cb2-afbe-b95db7a1c4be-0', tool_calls=[], invalid_tool_calls=[], usage_metadata={'input_tokens': 43, 'output_tokens': 28, 'total_tokens': 71})

In [26]:
from langchain_core.messages import AIMessage
model.invoke(
    [
        HumanMessage(content="Hi,my name is Anjali and I am a Agentic Ai Engineer"),
        AIMessage(content="Hello Anjali!  It's nice to meet you .\n As A AI Engineer ,What kind of work you do?"),
        HumanMessage(content="Hi What's my name and what do i do?")
]
)

# It remebers the previos context

AIMessage(content="Your name is Anjali, and you are an Agentic AI Engineer. \n\nIt seems like there might be a slight mistake with your profession. 'Agentic' is not a commonly used term in the field of AI or engineering. However, I'm assuming you meant to say 'Agentive' or possibly 'Artificial General Intelligence (AGI)' or 'AI Agent' as it relates to the concept of Artificial Intelligence where an AI system can perform tasks that typically require human intelligence such as decision-making, problem-solving, and reasoning.\n\nCan I help you with anything Anjali?", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 120, 'prompt_tokens': 97, 'total_tokens': 217, 'completion_time': 0.229505849, 'completion_tokens_details': None, 'prompt_time': 0.007344623, 'prompt_tokens_details': None, 'queue_time': 0.051689907, 'total_time': 0.236850472}, 'model_name': 'llama-3.1-8b-instant', 'system_fingerprint': 'fp_4387d3edbb', 'service_tier': 'on_demand', 'finish_reason': 

### Message History
We can use a Message History class to wrap ou model and make it stateful.This will keep track of inputs and outputs of the model, and store them in some datastore.Future interactions will then load those messages and pass them into the chian as part of the input.


In [27]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

store={}
def get_session_history(session_id:str)->BaseChatMessageHistory:
    if session_id not in store:
        store[session_id]=ChatMessageHistory()
    return store[session_id]

with_message_history=RunnableWithMessageHistory(model,get_session_history)

In [28]:
config={"configurable":{"session_id":"chat1"}}

In [29]:
response=with_message_history.invoke(
    [HumanMessage(content="Hi,my name is Anjali and I am a Agentic Ai Engineer")],
    config=config
    )

In [30]:
response.content

"Nice to meet you, Anjali.  As an Agentic AI Engineer, you must be working on creating intelligent systems that can take actions and make decisions on their own. That's a fascinating field. \n\nCan you tell me more about your current projects or areas of focus in Agentic AI? Are you working on developing autonomous agents, decision-making algorithms, or something else?"

In [31]:
#Change the config--> Change the Session id
config2={"configurable":{"session_id":"chat2"}}
response=with_message_history.invoke(
    [HumanMessage(content="What is my name?")],
    config=config2
    )
response.content

"I don't have any information about your name. I'm a large language model, I don't have personal interactions or memories, so I don't know who you are or what your name is. Each time you interact with me, it's a new conversation and I don't retain any information from previous conversations. If you'd like to share your name with me, I'd be happy to chat with you!"

### Prompt Templates
prompt Template help to turn a raw user information into a format that the llm can work with .In this case, the raw user input is just a message,which wee are passing to the LLM. 
Now we r doing this steps:
First , lets add in a system message with some custom instrucions (but still taking messages as input). Next,
wee'll add in more input besides just messages.

In [39]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

prompt=ChatPromptTemplate.from_messages(
    [
        ("system","You are a helpful assistance. Answer all the question to the best of your ability"),
        MessagesPlaceholder(variable_name="messages")
    ]
)

chain=prompt | model

In [35]:
chain.invoke({"messages":[HumanMessage(content="Hi My name is Anjali")]})

AIMessage(content="Hello Anjali. It's nice to meet you. Is there something I can help you with or would you like to chat?", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 28, 'prompt_tokens': 58, 'total_tokens': 86, 'completion_time': 0.028400727, 'completion_tokens_details': None, 'prompt_time': 0.002943805, 'prompt_tokens_details': None, 'queue_time': 0.054550775, 'total_time': 0.031344532}, 'model_name': 'llama-3.1-8b-instant', 'system_fingerprint': 'fp_ff2b098aaf', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None, 'model_provider': 'groq'}, id='lc_run--019c22ed-38c1-7ee3-8710-d48ca538366a-0', tool_calls=[], invalid_tool_calls=[], usage_metadata={'input_tokens': 58, 'output_tokens': 28, 'total_tokens': 86})

In [37]:
with_message_history=RunnableWithMessageHistory(chain,get_session_history)

In [38]:
config={"configurable":{"session_id":"chat3"}}
response= with_message_history.invoke(
    [HumanMessage(content="Hi My name is Anjali")],
    config=config,
)
response.content

"Hello Anjali, it's nice to meet you. How can I assist you today?"

In [40]:
## Add some more complexity 

prompt=ChatPromptTemplate.from_messages(
    [
        ("system","You are a helpful assistance. Answer all the question to the best of your ability in {language}"),
        MessagesPlaceholder(variable_name="messages")
    ]
)

chain=prompt | model

In [43]:
response=chain.invoke({"messages":[HumanMessage(content="Hi My name is Anjali")],"language":"Hindi"})
response.content

'नमस्ते अन्जलि! (Namaste Anjali!) मैं आपकी सहायता करने के लिए यहाँ हूँ। क्या आपके पास कोई प्रश्न या समस्या है जिससे मैं आपकी मदद कर सकता हूँ?'

There are multiple keys in the input, we need to specify the correct key to use to save the chat history

In [49]:

with_message_history=RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="messages"
)

In [50]:
from langchain_core.messages import HumanMessage

config = {"configurable": {"session_id": "chat4"}}

response = with_message_history.invoke(
    {
        "messages": [HumanMessage(content="Hi, My name is Anjali")],
        "language": "Hindi"
    },
    config=config
)

response.content


'नमस्ते अन्जलि! मैं आपकी मदद करने के लिए तैयार हूँ। आपके किस बारे में पूछना चाहती हैं?'

### Manage the conversational History

One important concept to understnad when building chatbots is how to manage conversation history.If left unmanaged , the list of messages will grow unbounded and potentially overflow the contect window of the LLM.Therefore, it is important to add a step that limits the size of the messages you are passing in.
<br/>

'trim_messages' : helper to reduce how many messages we are sending to the model.The trimmer allows us to specify how many tokens we want to keep,along with other parameters like if we want to always keep the system message and whether to allow partial messages.

In [68]:
from langchain_core.messages import SystemMessage,trim_messages

trimmer=trim_messages(
    max_tokens=30,
    strategy="last",
    token_counter=model,
    include_system=True,
    allow_partial=False,
    start_on="human"
)

messages=[
    SystemMessage(content="you're a good assistant"),
    HumanMessage(content="Hi!I am Anjali"),
    AIMessage(content="Hi!"),
    HumanMessage(content="I like vanilla icecream"),
    AIMessage(content="Nice"),
    HumanMessage(content="what is 2+2"),
    AIMessage(content="4"),
    HumanMessage(content="thanks"),
    AIMessage(content="No Problem!")
]


In [69]:
trimmer.invoke(messages)

[SystemMessage(content="you're a good assistant", additional_kwargs={}, response_metadata={}),
 HumanMessage(content='what is 2+2', additional_kwargs={}, response_metadata={}),
 AIMessage(content='4', additional_kwargs={}, response_metadata={}, tool_calls=[], invalid_tool_calls=[]),
 HumanMessage(content='thanks', additional_kwargs={}, response_metadata={}),
 AIMessage(content='No Problem!', additional_kwargs={}, response_metadata={}, tool_calls=[], invalid_tool_calls=[])]

In [70]:
from operator import itemgetter

from langchain_core.runnables import RunnablePassthrough
chain=(
    RunnablePassthrough.assign(messages=itemgetter("messages")|trimmer)
    | prompt
    | model
)
response=chain.invoke({
    "messages":messages + [HumanMessage(content="What ice cream do i like?")],
    "language":"English"
})
response.content

"That's a tough one, as I don't know your personal preferences. Could you give me a hint or tell me what types of flavors or ice cream textures you usually enjoy? That way, I can try to suggest some possibilities."

In [71]:
from operator import itemgetter

from langchain_core.runnables import RunnablePassthrough
chain=(
    RunnablePassthrough.assign(messages=itemgetter("messages")|trimmer)
    | prompt
    | model
)
response=chain.invoke({
    "messages":messages + [HumanMessage(content="What is 2+2 ?")],
    "language":"English"
})
response.content

'The answer is 4.'

In [86]:
from langchain_core.messages import HumanMessage

config = {"configurable": {"session_id": "chat6"}}

response = with_message_history.invoke(
    {
        "messages": [HumanMessage(content="Hi, My name is Anjali")],
        "language": "English"
    },
    config=config
)

response.content


"Nice to meet you, Anjali. I'm happy to assist you with any questions or topics you'd like to discuss. How are you today?"

In [87]:


response = with_message_history.invoke(
    {
        "messages": [HumanMessage(content="what math question did i ask?")],
        "language": "English"
    },
    config=config
)

response.content



"You didn't ask a math question yet, Anjali. Our conversation just started, and we were introduced. If you'd like to ask a math question, I'd be happy to help."